In [ ]:
# import libraries
import pandas as pd
import polars as pl
import io
import requests
import json
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

from src.config import TOKEN

: 

In [10]:
token = "TOKEN"
host = "https://firms.modaps.eosdis.nasa.gov/api/country/csv/"

source = "VIIRS_SNPP_NRT/"
country = "IDN/"
today = "2023-08-21"
day_range = "1"

url = (host + token + source + country + day_range + "/" + today)

get_data = reqs.get(url)

if get_data.status_code == 200:
    # csv_content = get_data.text
    csv_content = io.StringIO(get_data.text)
    viirs_df = pl.read_csv(csv_content)
    print(viirs_df)
else:
    print(f"Failed to fetch data, status code: {get_data.status_code}")

shape: (3, 1)
┌───────────────────────────────────┐
│ Invalid MAP_KEY.                  │
│ ---                               │
│ str                               │
╞═══════════════════════════════════╡
│ Invalid source.                   │
│ Invalid country code. Expects 3 … │
│ Invalid day range. Expects [1..1… │
└───────────────────────────────────┘


In [11]:
def fetch_viirs_data(today: str, day_range: str) -> pl.DataFrame:
    """
    Retrieves VIIRS active fires data from the NASA endpoint for a given date and range.

    Parameters:
    - today (str): The specific date for which the data is to be retrieved, in the format "YYYY-MM-DD".
    - day_range (str): The range of days for which the data is to be retrieved.

    Returns:
    - pl.DataFrame: A Polars DataFrame containing the VIIRS data if successful, or None if an error occurred.

    Examples:
    >>> get_viirs_data("2023-08-19", "1")
    """

    try:
        token = "TOKEN"
        host = "https://firms.modaps.eosdis.nasa.gov/api/country/csv/"
        source = "VIIRS_NOAA20_NRT/"
        country = "IDN/"

        url = (host + token + source + country + day_range + "/" + today)

        get_data = requests.get(url)
        get_data.raise_for_status() # Raises an HTTPError if the HTTP request returned an unsuccessful status code

        csv_content = io.StringIO(get_data.text)
        viirs_df = pl.read_csv(csv_content)
        
        return viirs_df

    except requests.RequestException as e:
        print(f"An error occurred while making the HTTP request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [12]:
today = "2023-08-19"
day_range = "1"

viirs_df = fetch_viirs_data(today, day_range)

print(viirs_df)

shape: (3, 1)
┌───────────────────────────────────┐
│ Invalid MAP_KEY.                  │
│ ---                               │
│ str                               │
╞═══════════════════════════════════╡
│ Invalid source.                   │
│ Invalid country code. Expects 3 … │
│ Invalid day range. Expects [1..1… │
└───────────────────────────────────┘
